# Importing Libraries

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 12.3 MB/s eta

In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 34.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install tokenizers

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.2 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from time import time
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

# Configuring Model

In [ ]:
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get('Rag')
login(token=hf_token)
model_id = "meta-llama/Llama-2-7b-chat-hf"
device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoTokenizer

time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Prepare model, tokenizer: 77.168 sec.


# Text Generation Pipeline

In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


Prepare pipeline: 0.41 sec.


In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:
test_model(tokenizer,query_pipeline,"Using these documents, answer the user’s question succinctly. Keep it in 100 words.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Test inference: 14.068 sec.
Result: Using these documents, answer the user’s question succinctly. Keep it in 100 words.
The documents are:

1. A letter from a doctor stating that the user has been suffering from depression for the past year and has been undergoing treatment.
2. A copy of the user’s medical insurance card.
3. A copy of the user’s employment contract.

User’s question: Can I take a mental health day without worrying about losing my job?

Answer: As per your employment contract, you are entitled to take up to 3 mental health days per year. Please consult with your HR department for more details. Additionally, your medical insurance card covers mental health treatments, including therapy sessions. Consult with your doctor for more information.


# RAG Building

In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
llm(prompt="Using these documents, answer the user’s question succinctly. Keep it in 100 words.")

/tmp/ipython-input-2324538616.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipeline)
/tmp/ipython-input-2324538616.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm(prompt="Using these documents, answer the user’s question succinctly. Keep it in 100 words.")


'Using these documents, answer the user’s question succinctly. Keep it in 100 words.\n\nUser: Can I get a copy of my W-2 form from last year?\n\nDocument 1: The IRS website provides instructions on how to obtain a copy of your W-2 form from last year.\n\nDocument 2: You can also contact your employer directly to request a copy of your W-2 form from last year.\n\nDocument 3: If you are unable to obtain a copy of your W-2 form from your employer or the IRS, you may need to file for a duplicate W-2 form.\n\nAnswer: Yes, you can obtain a copy of your W-2 form from last year by visiting the IRS website or contacting your employer directly. If you are unable to obtain a copy, you may need to file for a duplicate W-2 form.'

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving A study on predicting surface solar.txt to A study on predicting surface solar.txt


In [ ]:
import os
os.listdir("/content")


['.config',
 'A study on predicting surface solar.txt',
 '.ipynb_checkpoints',
 'sample_data']

In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
txt_loader = DirectoryLoader("/content", glob="*.txt", loader_cls=TextLoader)
pdf_loader = DirectoryLoader("/content", glob="*.pdf", loader_cls=PyPDFLoader)


docs = txt_loader.load() + pdf_loader.load()
print("Total documents loaded:", len(docs))


Total documents loaded: 1


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/tmp/ipython-input-2257289084.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

In [ ]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [ ]:
import pickle
with open('rag_search_model.pkl', 'wb') as f:
    pickle.dump(qa, f)

In [ ]:
query = "What were the main topics in the document? Summarize. Keep it under 50 words."
test_rag(qa, query)

Query: What were the main topics in the document? Summarize. Keep it under 50 words.



> Entering new RetrievalQA chain...


/tmp/ipython-input-3897161138.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)



> Finished chain.
Inference time: 6.134 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

A study on predicting surface solar radiation using a deep learning approach, combining Convolutional Neural Networks (CNNs) with Long Short-Term Memory (LSTM) and Gated Recurrent Unit (GRU) networks to improve prediction accuracy and Adaptive Variance Optimization for reducing the loss function. 
​ 
The model is validated using data from NASA's Solar Radiation and Climate Experiment (SORCE) dataset. 
​​
Reviews traditional and deep learning methods for solar radiation forecasting, highlighting the limitations of statistical models and the advantages of deep learning. 
​
Proposed Method: Describes the data acquisition, preprocessing, feature extraction, and the architecture of the hybrid CNN-LSTM/GRU model. 
​The Adaptive Variance Optimization technique is used for feat

In [ ]:
query = "Summarize. Keep it under 100 words."
test_rag(qa, query)

Query: Summarize. Keep it under 100 words.



> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 9.642 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

A study on predicting surface solar radiation using a deep learning approach, combining Convolutional Neural Networks (CNNs) with Long Short-Term Memory (LSTM) and Gated Recurrent Unit (GRU) networks to improve prediction accuracy and Adaptive Variance Optimization for reducing the loss function. 
​ 
The model is validated using data from NASA's Solar Radiation and Climate Experiment (SORCE) dataset. 
​​
Reviews traditional and deep learning methods for solar radiation forecasting, highlighting the limitations of statistical models and the advantages of deep learning. 
​
Proposed Method: Describes the data acquisition, preprocessing, feature extraction, and the architecture of the hybrid C

In [ ]:
query = "Is there any mission where nasa and isro are collabrating together. Keep it under 100 words."
test_rag(qa, query)

Query: Is there any mission where nasa and isro are collabrating together. Keep it under 100 words.



> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 10.417 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

A study on predicting surface solar radiation using a deep learning approach, combining Convolutional Neural Networks (CNNs) with Long Short-Term Memory (LSTM) and Gated Recurrent Unit (GRU) networks to improve prediction accuracy and Adaptive Variance Optimization for reducing the loss function. 
​ 
The model is validated using data from NASA's Solar Radiation and Climate Experiment (SORCE) dataset. 
​​
Reviews traditional and deep learning methods for solar radiation forecasting, highlighting the limitations of statistical models and the advantages of deep learning. 
​
Proposed Method: Describes the data acquisition, preprocessing